In [1]:
# from langchain import OpenAI
from langchain_openai.llms import OpenAI
from langchain.prompts import PromptTemplate
import os
from dotenv import load_dotenv

load_dotenv()

# client = OpenAI.Client()
OpenAI.api_key = os.getenv("OPENAI_API_KEY")
llm = OpenAI()

prompt_template = PromptTemplate.from_template("""
    Responda a seguinte pergunta do usuário:
    {pergunta}
""")


In [2]:
print(prompt_template.format(pergunta="O que é um SaaS?"))


    Responda a seguinte pergunta do usuário:
    O que é um SaaS?



In [3]:
prompt_template = PromptTemplate.from_template("""
    Responda a seguinte pergunta do  em até {n_palavras} palavras:
    {pergunta}
""")


In [4]:
print(prompt_template.format(pergunta="O que é um SaaS?", n_palavras=15))


    Responda a seguinte pergunta do  em até 15 palavras:
    O que é um SaaS?



In [5]:
prompt_template = PromptTemplate.from_template("""
    Responda a seguinte pergunta do  em até {n_palavras} palavras:
    {pergunta}
""", partial_variables={"n_palavras": 10})

In [6]:
print(prompt_template.format(pergunta="O que é LangChain?"))


    Responda a seguinte pergunta do  em até 10 palavras:
    O que é LangChain?



### Utilizando Múltiplos Prompts

In [7]:
from langchain.prompts import PromptTemplate

template_word_count = PromptTemplate.from_template("""
Responda a seguinte pergunta em até {n_palavras} palavras:
""")

template_line_count = PromptTemplate.from_template("""
Responda a seguinte pergunta em até {n_linhas} linhas:
""")

template_idioma = PromptTemplate.from_template("""
Responda a resposta em {idioma}
""")

template_final = (template_word_count + template_line_count + template_idioma +
                 "Responda a pergunta seguindo as instruções {pergunta}")

In [8]:
print(template_final)

input_variables=['idioma', 'n_linhas', 'n_palavras', 'pergunta'] template='\nResponda a seguinte pergunta em até {n_palavras} palavras:\n\nResponda a seguinte pergunta em até {n_linhas} linhas:\n\nResponda a resposta em {idioma}\nResponda a pergunta seguindo as instruções {pergunta}'


In [9]:
prompt_final = template_final.format(n_palavras=10, n_linhas=1, idioma="Inglês", pergunta="O que é o sol?")
llm.invoke(prompt_final)

'\n\nWhat is the sun?'

In [13]:
print(prompt_final)


Responda a seguinte pergunta em até 10 palavras:

Responda a seguinte pergunta em até 1 linhas:

Responda a resposta em Inglês
Responda a pergunta seguindo as instruções O que é o sol?


### Templates para Chat

In [14]:
from langchain.prompts import ChatPromptTemplate

chat_template = ChatPromptTemplate.from_template("Essa é minha dúvida: {duvida}")
chat_template.format_messages(duvida="Quem é você?")

[HumanMessage(content='Essa é minha dúvida: Quem é você?')]

In [15]:
from langchain.prompts import ChatPromptTemplate

chat_template = ChatPromptTemplate.from_messages(
    [
        ("system", "Você é um assistente irônico e se chama {nome_assistente}"),
        ("human", "Olá como vai?"),
        ("ai", "Estou bem, como posso lhe ajudar?"),
        ("human", "{pergunta}")
    ]
)


In [16]:
chat_template

ChatPromptTemplate(input_variables=['nome_assistente', 'pergunta'], messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['nome_assistente'], template='Você é um assistente irônico e se chama {nome_assistente}')), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='Olá como vai?')), AIMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='Estou bem, como posso lhe ajudar?')), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['pergunta'], template='{pergunta}'))])

In [17]:
chat_template.format_messages(nome_assistente="BotX", pergunta="Qual o seu nome?")

[SystemMessage(content='Você é um assistente irônico e se chama BotX'),
 HumanMessage(content='Olá como vai?'),
 AIMessage(content='Estou bem, como posso lhe ajudar?'),
 HumanMessage(content='Qual o seu nome?')]

In [18]:
from langchain_openai import ChatOpenAI

chat = ChatOpenAI()
chat.invoke(chat_template.format_messages(nome_assistente="BotX", pergunta="Qual o seu nome?"))

AIMessage(content='Meu nome é BotX, estou aqui para responder suas perguntas com um toque de ironia. Como posso te ajudar?', response_metadata={'token_usage': {'completion_tokens': 30, 'prompt_tokens': 55, 'total_tokens': 85, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-c1478cb2-c056-455b-863e-b5d8f853b4a1-0')

### Few Shot Prompting

In [23]:
exemplos = [
    {
        "pergunta": "Qual é a maior montanha do mundo, o Monte Everest ou o K2?",
        "resposta": """
São necessárias perguntas de acompanhamento aqui: Sim.
Pergunta de acompanhamento: Qual é a altura do Monte Everest?
Resposta intermediária: O Monte Everest tem 8.848 metros de altura.
Pergunta de acompanhamento: Qual é a altura do K2?
Resposta intermediária: O K2 tem 8.611 metros de altura.
Então a resposta final é: Monte Everest
""",
    },
    {
        "pergunta": "Quem nasceu primeiro, Charles Darwin ou Albert Einstein?",
        "resposta": """
São necessárias perguntas de acompanhamento aqui: Sim.
Pergunta de acompanhamento: Quando nasceu Charles Darwin?
Resposta intermediária: Charles Darwin nasceu em 12 de fevereiro de 1809.
Pergunta de acompanhamento: Quando nasceu Albert Einstein?
Resposta intermediária: Albert Einstein nasceu em 14 de março de 1879.
Então a resposta final é: Charles Darwin
""",
    },
    {
        "pergunta": "Quem foi o pai de Napoleão Bonaparte?",
        "resposta": """
São necessárias perguntas de acompanhamento aqui: Sim.
Pergunta de acompanhamento: Quem foi Napoleão Bonaparte?
Resposta intermediária: Napoleão Bonaparte foi um líder militar e imperador francês.
Pergunta de acompanhamento: Quem foi o pai de Napoleão Bonaparte?
Resposta intermediária: O pai de Napoleão Bonaparte foi Carlo Bonaparte.
Então a resposta final é: Carlo Bonaparte
""",
    },
    {
        "pergunta": "Os filmes 'O Senhor dos Anéis' e 'O Hobbit' foram dirigidos pelo mesmo diretor?",
        "resposta": """
São necessárias perguntas de acompanhamento aqui: Sim.
Pergunta de acompanhamento: Quem dirigiu 'O Senhor dos Anéis'?
Resposta intermediária: 'O Senhor dos Anéis' foi dirigido por Peter Jackson.
Pergunta de acompanhamento: Quem dirigiu 'O Hobbit'?
Resposta intermediária: 'O Hobbit' também foi dirigido por Peter Jackson.
Então a resposta final é: Sim
""",
    },
]


In [24]:
from langchain.prompts.few_shot import FewShotPromptTemplate
from langchain.prompts.prompt import PromptTemplate

example_prompt = PromptTemplate(
    input_variables=["pergunta", "resposta"],
    template = "Pergunta: {pergunta}\nResposta: {resposta}"
)

In [25]:
example_prompt.format(**exemplos[0])


'Pergunta: Qual é a maior montanha do mundo, o Monte Everest ou o K2?\nResposta: \nSão necessárias perguntas de acompanhamento aqui: Sim.\nPergunta de acompanhamento: Qual é a altura do Monte Everest?\nResposta intermediária: O Monte Everest tem 8.848 metros de altura.\nPergunta de acompanhamento: Qual é a altura do K2?\nResposta intermediária: O K2 tem 8.611 metros de altura.\nEntão a resposta final é: Monte Everest\n'

In [26]:
prompt = FewShotPromptTemplate(
    examples = exemplos,
    example_prompt = example_prompt,
    suffix = "Pergunta: {input}",
    input_variables = ["input"]
)

In [27]:
print(prompt.format(input="Quem é melhor Messi ou Cristiano Ronaldo?"))

Pergunta: Qual é a maior montanha do mundo, o Monte Everest ou o K2?
Resposta: 
São necessárias perguntas de acompanhamento aqui: Sim.
Pergunta de acompanhamento: Qual é a altura do Monte Everest?
Resposta intermediária: O Monte Everest tem 8.848 metros de altura.
Pergunta de acompanhamento: Qual é a altura do K2?
Resposta intermediária: O K2 tem 8.611 metros de altura.
Então a resposta final é: Monte Everest


Pergunta: Quem nasceu primeiro, Charles Darwin ou Albert Einstein?
Resposta: 
São necessárias perguntas de acompanhamento aqui: Sim.
Pergunta de acompanhamento: Quando nasceu Charles Darwin?
Resposta intermediária: Charles Darwin nasceu em 12 de fevereiro de 1809.
Pergunta de acompanhamento: Quando nasceu Albert Einstein?
Resposta intermediária: Albert Einstein nasceu em 14 de março de 1879.
Então a resposta final é: Charles Darwin


Pergunta: Quem foi o pai de Napoleão Bonaparte?
Resposta: 
São necessárias perguntas de acompanhamento aqui: Sim.
Pergunta de acompanhamento: Quem 

In [28]:
llm.invoke(prompt.format(input="Quem fez mais gols Messi ou Cristiano Ronaldo?"))

'\nResposta: \nSão necessárias perguntas de acompanhamento aqui: Sim.\nPergunta de acompanhamento: Quantos gols Messi fez?\nResposta intermediária: Messi fez 731 gols em sua carreira profissional até o momento.\nPergunta de acompanhamento: Quantos gols Cristiano Ronaldo fez?\nResposta intermediária: Cristiano Ronaldo fez 745 gols em sua carreira profissional até o momento.\nEntão a resposta final é: Cristiano Ronaldo (745 gols)'

### Few Shot Prompting em Chat

In [ ]:
from langchain_openai import ChatOpenAI

chat = ChatOpenAI()


In [33]:
from langchain.prompts.few_shot import FewShotChatMessagePromptTemplate
from langchain.prompts import ChatPromptTemplate

example_prompt = ChatPromptTemplate.from_messages(
    [
        ("human", "{pergunta}"),
        ("ai", "{resposta}")
    ]
)

print(example_prompt.format_messages(**exemplos[0]))


[HumanMessage(content='Qual é a maior montanha do mundo, o Monte Everest ou o K2?'), AIMessage(content='\nSão necessárias perguntas de acompanhamento aqui: Sim.\nPergunta de acompanhamento: Qual é a altura do Monte Everest?\nResposta intermediária: O Monte Everest tem 8.848 metros de altura.\nPergunta de acompanhamento: Qual é a altura do K2?\nResposta intermediária: O K2 tem 8.611 metros de altura.\nEntão a resposta final é: Monte Everest\n')]


In [34]:
few_shot_template = FewShotChatMessagePromptTemplate(
    examples=exemplos,
    example_prompt=example_prompt
)

prompt_final = ChatPromptTemplate.from_messages([
    few_shot_template,
    ("human", "{input}")
])

prompt = prompt_final.format_messages(input="Quem fez mais gols, Messi ou Cristiano Ronaldo?")


In [35]:
chat.invoke(prompt)

AIMessage(content='Até o momento, Cristiano Ronaldo marcou mais gols do que Lionel Messi em suas carreiras. Cristiano Ronaldo é conhecido por sua incrível capacidade de marcar gols e tem tido um desempenho excepcional ao longo dos anos, ultrapassando Messi em número de gols marcados. No entanto, é importante ressaltar que ambos são considerados dois dos maiores jogadores de futebol de todos os tempos e têm tido carreiras impressionantes.', response_metadata={'token_usage': {'completion_tokens': 108, 'prompt_tokens': 504, 'total_tokens': 612, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-43a5f19f-9e14-4856-acb6-f231b9ea1a0a-0')